In [2]:
import pandas as pd 

# Read raw data from excel file
raw_data = pd.read_excel('../data/raw/sanitary_registry_medicines.xlsx')

# Rename columns and drop columns that are not needed
columns_to_drop = ['Solicitud', 'Nombre Generico', 'Principio Activo', 'Distritbuidora', 'Representante', 'Empaque', 'Especificacion_envase_primario']
columns_to_rename = {'Registro Sanitario': 'sanitary_registry', 'NombreProducto': 'product_name', 'Fabricante' : 'manufacturer', 'Forma Farmaceutica': 'pharmaceutical_form', 'Fecha Expiracion': 'expiration_date', 'Estado': 'status', 'Cantidad': 'quantity', 'Titular Registro': 'holder', 'Fecha Registro': 'register_date', 'Fecha Vencimiento': 'expiration_date', 'Estatus' : 'status', 'Tipo de producto': 'product_type' }


raw_data.drop( columns_to_drop, axis=1, inplace=True)
raw_data.rename(columns=columns_to_rename, inplace=True)

# Drop rows with null values
raw_data.dropna(inplace=True)


raw_data = raw_data[raw_data['manufacturer'] != 'N/D']
raw_data = raw_data[raw_data['product_type'] == 'MEDICAMENTOS']
raw_data = raw_data[raw_data['product_name'] != 'N/D']
raw_data = raw_data[raw_data['expiration_date'] != 'N/D']
raw_data = raw_data[raw_data['status'] != 'N/D']
raw_data = raw_data[raw_data['status'] != 'NO VIGENTE']
raw_data = raw_data[raw_data['pharmaceutical_form'] != 'N/D']



In [3]:


# Transform data types of columns and parse expiration date
raw_data['sanitary_registry'].astype('str')
raw_data['product_name'].to_string(index=False)
raw_data['manufacturer'].to_string(index=False)
raw_data['pharmaceutical_form'].to_string(index=False)
raw_data['expiration_date'] = pd.to_datetime(raw_data['expiration_date'], infer_datetime_format=True)

# Create a regex to extract the year from the sanitary registry (str | int) like 2019-00000 or SUB2019-00000
year_of_register_pattern = r'(\b(?:19[5-9]\d|20[0-4]\d|2050)\b|\b(?:19[5-9]\d|20[0-4]\d|2050)(?:(?<=\b)-\d+))?\b'

raw_data['register_year'] = raw_data['sanitary_registry'].str.extract(year_of_register_pattern)
raw_data.dropna(inplace=True)
raw_data['register_year'] = raw_data['register_year'].astype('int64')


# Durability of the product in years

raw_data['durability'] =  raw_data['expiration_date'].dt.year - raw_data['register_year']

# Drop unnecessary columns

raw_data.drop(columns=['status','product_type','sanitary_registry'], inplace=True)




C:\Users\Carlos Olivo\AppData\Local\Temp\ipykernel_28516\3258210838.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  raw_data['expiration_date'] = pd.to_datetime(raw_data['expiration_date'], infer_datetime_format=True)


In [4]:

def extract_singular(x):
    if x.endswith('ES'):
        return x[:-2]
    elif x.endswith('S'):
        return x[:-1]
    else:
        return x
    
# Normalize product names

raw_data['product_name'] = raw_data['product_name'].str.lower()

raw_data['product_name'] = raw_data['product_name'].str.replace('"', '')


# Normalize manufacturer names
raw_data['manufacturer'] = raw_data['manufacturer'].str.split(',').str[0]
raw_data['manufacturer'] = raw_data['manufacturer'].str.lower()
raw_data['manufacturer'] = raw_data['manufacturer'].str.replace('"', '')
raw_data['manufacturer'] = raw_data['manufacturer'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
raw_data['manufacturer'] = raw_data['manufacturer'].str.strip()

# Normalize pharmaceutical form

raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.upper()
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.replace('"', '')
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.strip()
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.split(' ').str[0]
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].str.split('/').str[0]
raw_data['pharmaceutical_form'] = raw_data['pharmaceutical_form'].apply(extract_singular)

# Save clean data to csv file
raw_data.sort_values(by=['register_year'], inplace=True)
raw_data.to_csv('../data/processed/sanitary_registry_medicines.csv', index=False)